# Dispatch with Energy Storage

In this case, we will show the usage of energy storage included dispatch.

In AMS, ``ESD1`` is a dispatch model for energy storage, which has a corresponding
dynamic model ``ESD1`` in ANDES.

In [1]:
import pandas as pd

import ams

In [2]:
ams.config_logger(stream_level=20)

A small-size PJM 5-bus case with ESD1 is used in this example.

In [3]:
sp = ams.load(ams.get_case('5bus/pjm5bus_demo.xlsx'),
              setup=True,
              no_output=True)

Parsing input file "/Users/jinningwang/work/ams/ams/cases/5bus/pjm5bus_demo.xlsx"...
Input file parsed in 0.0954 seconds.
Zero Line parameters detected, adjusted to default values: rate_b, rate_c.
Parameters c2, c1 are altered to 0 as they are associated with ESD1 for following GCost: GCost_5
All bus type are PQ, adjusted given load and generator connection status.
System set up in 0.0022 seconds.


The model parameters can be inspected as follow.

In [4]:
sp.ESD1.as_df()

,idx,u,name,bus,gen,Sn,gammap,gammaq,SOCmin,SOCmax,...,EtaC,EtaD,cesdc,cesdd,tdc,tdd,tdc0,tdd0,ucd0,udd0
uid,,,,,,,,,,,,,,,,,,,,,
0,ESD1_1,1.0,ESD1_1,1,PV_2,100.0,1.0,1.0,0.1,0.9,...,1.0,1.0,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0


`RTEDES` extends RTED to include energy storage.

Note that mixed integer linear programming (MILP) requires
capable solvers such as Gurobi or CPLEX.
They might require extra installation and have their own license.

The example here only aims to show the usage of RTEDES.
More details can be found at [CVXPY - Choosing a solver](https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver).

In [5]:
sp.RTEDES.run(solver='SCIP')

Building system matrices
Parsing OModel for <RTEDES>
Evaluating OModel for <RTEDES>
Finalizing OModel for <RTEDES>
<RTEDES> initialized in 0.0179 seconds.
<RTEDES> solved as optimal in 0.0298 seconds, converged in -1 iteration with SCIP.


True

Note that, in RTED, the time interval is 5/60 [H] by default, and the
dispatch model has been adjusted accordingly.

In [6]:
sp.RTEDES.SOC.v

array([0.59641411])

Routine ``RTEDEPS`` is the price run of RTED with ESD1 model, where the binary terms are substituted by their solutions from the ``RTEDES`` or ``RTEDES2``

In [7]:
sp.RTEDESP.run(solver='CLARABEL')

<RTEDESP>: ESD1 associated StaticGen.p0 has been setaccording to <RTEDES>
Parsing OModel for <RTEDESP>
Evaluating OModel for <RTEDESP>
Finalizing OModel for <RTEDESP>
<RTEDESP> initialized in 0.0287 seconds.
<RTEDESP> solved as optimal in 0.0346 seconds, converged in 11 iterations with CLARABEL.


True

After it, you can have the LMP:

In [8]:
sp.RTEDESP.pi.v

array([0.01221973, 0.00833333, 0.025     , 0.01678462, 0.01302902])

Similarly, multi-period dispatch ``EDES`` and ``UCES`` are also available.
They have 1 [H] time interval by default.

In [9]:
sp.EDES.config.t

1

In [10]:
sp.EDES.run(solver='SCIP')

Parsing OModel for <EDES>
Evaluating OModel for <EDES>
Finalizing OModel for <EDES>
<EDES> initialized in 0.0230 seconds.
<EDES> solved as optimal in 0.3200 seconds, converged in -1 iteration with SCIP.


True

In [11]:
sp.EDES.SOC.v

array([[0.61443053, 0.63070058, 0.64861129, 0.66726776, 0.68632196,
        0.70507786, 0.72164621, 0.73562929, 0.74831972, 0.75986667,
        0.76936667, 0.77646667, 0.78146667, 0.75546793, 0.71489114,
        0.67328647, 0.63070864, 0.58813081, 0.54622671, 0.50662611,
        0.46707564, 0.47347564, 0.48512202, 0.5       ]])

In [12]:
sp.UCES.run(solver='SCIP')

All generators are online at initial, make initial guess for commitment.
As initial commitment guess, turn off StaticGen: PV_1
Parsing OModel for <UCES>
Evaluating OModel for <UCES>
Finalizing OModel for <UCES>
<UCES> initialized in 0.0226 seconds.
<UCES> solved as optimal in 0.2469 seconds, converged in -1 iteration with SCIP.


True

In [13]:
sp.UCES.SOC.v

array([[0.59882146, 0.59882146, 0.59882146, 0.59882146, 0.59882146,
        0.59882146, 0.59882146, 0.59672403, 0.59197199, 0.59197199,
        0.58252336, 0.57062432, 0.55658119, 0.54581983, 0.54581983,
        0.54581983, 0.54571032, 0.54560081, 0.54560081, 0.52777997,
        0.51261374, 0.5       , 0.5       , 0.5       ]])